Olá, Paulo

Meu nome é Ramon. Ao ao longo do texto farei algumas observações sobre melhorias no código. Nas partes em que não houver comentários, está tudo certo, ok? Estarei aberto a feedbacks e discussões sobre o tema.

**Peço que mantenha e não altere os comentários que eu fizer por aqui para que possamos nos localizar posteriormente, ok?**

Mais uma coisa, vamos utilizar um código de cores para você entender os meus feedbacks no seu notebook. Funciona assim:

<div class="alert alert-block alert-success">
<b> Comentário do revisor: </b> <a class="tocSkip"></a>

Sucesso. Tudo foi feito corretamente.
</div>

<div class="alert alert-block alert-warning">
<b>Comentário do revisor: </b> <a class="tocSkip"></a>

Alerta não crítco, mas que pode ser corrigido para melhoria geral no seu código/análise
</div>

<div class="alert alert-block alert-danger">

<b>Comentário do revisor: </b> <a class="tocSkip"></a>
    
Erro que precisa ser consertado, caso contrário seu projeto **não** será aceito.
</div>

Você pode me responder usando isso:
<div class="alert alert-block alert-info">
<b>Resposta do Aluno.</b> <a class="tocSkip"></a>
</div>

###

<div class="alert alert-block alert-success">
<b>Comentário Geral do Revisor</b> <a class="tocSkip"></a>

Obrigado por enviar seu projeto. 
    
Parabéns, você fez um bom trabalho.

Como pontos fortes do projeto, destaco:
- Código simples e eficiente (sem loop desnecessário)
- Uso adequado dos recursos da lição

    
<span class="text-danger">Porém, há espaço para melhorias: </span>
 * <span class="text-danger"> ajustar codificação de variáveis, ajustar calibração de hiperparâmetros </span>


Portanto, sugiro que você corrija os pontos indicados.
    
Se você tiver alguma dúvida sobre de algo que você não tenha entendido, sinta-se à vontade para perguntar ao tutor ou deixar um comentário em **azul** para que, na próxima iteração, eu possa responder.

Conte conosco. Lembre-se de que você pode tirar dúvidas com revisores usando os comentários ou buscar ajuda de um tutor.
Bom trabalho. 


</div>



<div class="alert alert-block alert-danger">
<b> Comentário do revisor V2: </b> <a class="tocSkip"></a>

- O projeto voltou com erro de execução
- Para aprovação, precisa ajustar a parte de calibração de hiperparâmetros e o tratamento de nans
- Essa parte de calibração é muito importante. Como nossa comunicação é limitada, você deseja tutoria? Não podemos calibrar hiperparãmetros no conjunto de teste. Ou usamos o conjunto validação ou usamos validação cruzada no conjunto de treino.
</div>


O serviço de vendas de carros usados Rusty Bargain está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos: especificações técnicas, versões de acabamento e preços. Você precisa construir o modelo para determinar o valor. 

Rusty Bargain está interessado em:

- a qualidade da predição;
- a velocidade da predição;
- o tempo necessário para o treinamento

## Preparação de Dados

In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor

# Carregar os dados
data = pd.read_csv("/datasets/car_data.csv")

# Visualizar os dados
print(data.head(), "\n")
print("\n", data.info())


        DateCrawled  Price VehicleType  RegistrationYear Gearbox  Power  \
0  24/03/2016 11:52    480         NaN              1993  manual      0   
1  24/03/2016 10:58  18300       coupe              2011  manual    190   
2  14/03/2016 12:52   9800         suv              2004    auto    163   
3  17/03/2016 16:54   1500       small              2001  manual     75   
4  31/03/2016 17:25   3600       small              2008  manual     69   

   Model  Mileage  RegistrationMonth  FuelType       Brand NotRepaired  \
0   golf   150000                  0    petrol  volkswagen         NaN   
1    NaN   125000                  5  gasoline        audi         yes   
2  grand   125000                  8  gasoline        jeep         NaN   
3   golf   150000                  6    petrol  volkswagen          no   
4  fabia    90000                  7  gasoline       skoda          no   

        DateCreated  NumberOfPictures  PostalCode          LastSeen  
0  24/03/2016 00:00               

In [ ]:
# Descartar Features irrelevantes para o preço do carro
data = data.drop(columns = ['PostalCode', 'NumberOfPictures', 'DateCrawled', 'RegistrationYear', 'RegistrationMonth', 'DateCreated', 'LastSeen'])
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Price        354369 non-null  int64 
 1   VehicleType  316879 non-null  object
 2   Gearbox      334536 non-null  object
 3   Power        354369 non-null  int64 
 4   Model        334664 non-null  object
 5   Mileage      354369 non-null  int64 
 6   FuelType     321474 non-null  object
 7   Brand        354369 non-null  object
 8   NotRepaired  283215 non-null  object
dtypes: int64(3), object(6)
memory usage: 24.3+ MB
None


In [ ]:
data.sample(3)


,Price,VehicleType,Gearbox,Power,Model,Mileage,FuelType,Brand,NotRepaired
67244,1750,sedan,manual,150,5er,150000,petrol,bmw,no
317924,350,coupe,NaN,160,NaN,5000,petrol,volkswagen,NaN
146254,6499,coupe,manual,231,rx_reihe,150000,petrol,mazda,no


In [ ]:
data.isna().sum()


Price              0
VehicleType    37490
Gearbox        19833
Power              0
Model          19705
Mileage            0
FuelType       32895
Brand              0
NotRepaired    71154
dtype: int64

In [ ]:
columns_to_fix = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']
for col in data.columns:
    if col in columns_to_fix:
        data[col] = data[col].fillna('unknown')
data.isna().sum()


Price          0
VehicleType    0
Gearbox        0
Power          0
Model          0
Mileage        0
FuelType       0
Brand          0
NotRepaired    0
dtype: int64

<div class="alert alert-block alert-danger">
<b> Comentário do revisor V2: </b> <a class="tocSkip"></a>

- eu adicionei PostalCode e  NumberOfPictures no .drop() pois não não é relevante
- note que há nans. Como vocẽ pretende tratar esses missing data? Por exemplo, substituir por 'ausente' caso seja categórico, pela mediana, outros por 0 (ex: notrepaired)
</div>


In [10]:
# Usaremos One-Hot-Encoding para tratar variáveis categóricas
categorical_features = []
for col in data.columns:
    if data[col].dtype == object:
        categorical_features.append(col)

data_encoded = data.copy()
data_encoded = pd.get_dummies(data_encoded, columns=categorical_features, drop_first=True)
print(data_encoded.head())



   Price  Power  Mileage  VehicleType_convertible  VehicleType_coupe  \
0    480      0   150000                        0                  0   
1  18300    190   125000                        0                  1   
2   9800    163   125000                        0                  0   
3   1500     75   150000                        0                  0   
4   3600     69    90000                        0                  0   

   VehicleType_other  VehicleType_sedan  VehicleType_small  VehicleType_suv  \
0                  0                  0                  0                0   
1                  0                  0                  0                0   
2                  0                  0                  0                1   
3                  0                  0                  1                0   
4                  0                  0                  1                0   

   VehicleType_unknown  ...  Brand_smart  Brand_sonstige_autos  Brand_subaru  \
0           

<div class="alert alert-block alert-danger">
<b> Comentário do revisor V2: </b> <a class="tocSkip"></a>

erro de execução de variável naõ definida
</div>


## Treinamento do modelo

In [ ]:
def find_best_hyperparameters(model, features_train, target_train):
    param_grids = {
        "linearregression": [
            {"fit_intercept": True},
        ],
        "randomforestregressor": [
            {"n_estimators": 50, "max_depth": 10, "n_jobs": -1},
            {"n_estimators": 100, "max_depth": 15, "n_jobs": -1},
            {"n_estimators": 100, "max_depth": 8, "n_jobs": -1},
        ],
        "lgbmregressor": [
            {
                "n_estimators": 50,
                "learning_rate": 0.1,
                "max_depth": 8,
                "num_leaves": 31,
                "n_jobs": -1,
                "verbosity": -1,
            },
            {
                "n_estimators": 100,
                "learning_rate": 0.1,
                "max_depth": 10,
                "num_leaves": 50,
                "n_jobs": -1,
                "verbosity": -1,
            },
            {
                "n_estimators": 80,
                "learning_rate": 0.15,
                "max_depth": 6,
                "num_leaves": 64,
                "n_jobs": -1,
                "verbosity": -1,
            },
        ],
    }

    model_name = type(model).__name__.lower()
    param_grid = param_grids.get(model_name, [])

    if len(param_grid) == 0:
        print(f"Nenhum hiperparâmetro para calibrar em {type(model).__name__}")
        return None

    best_mse = float("inf")
    best_params = None

    print(f"Calibrando hiperparâmetros para {type(model).__name__}...")
    
    for param_set in param_grid:
        model.set_params(**param_set)
        cv_scores = cross_val_score(
            model, features_train, target_train, 
            cv=5, scoring='neg_mean_squared_error'
        )
        cv_mse = -cv_scores.mean()
        
        if cv_mse < best_mse:
            best_mse = cv_mse
            best_params = param_set
    
    print(f"Melhores parâmetros: {best_params}")
    print(f"MSE na validação cruzada: {best_mse:.2f}")
    return best_params, best_mse


In [ ]:

def evaluate_final_model(model, best_params, features_train, target_train, 
                        features_test, target_test):
    if best_params:
        model.set_params(**best_params)

    # Treinar no conjunto completo de treinamento
    model.fit(features_train, target_train)

    # Realizar previsões com o conjunto de validação
    predictions = model.predict(features_test)

    # Avaliar modelo
    mse = mean_squared_error(target_test, predictions)
    r2 = r2_score(target_test, predictions)

    print(f"MSE no conjunto de teste: {mse:.2f}")
    print(f"R2 no conjunto de teste: {r2:.2f}")    

    return mse, r2


## Análise do modelo

In [17]:
# Dividir os dados em treinamento e teste
X = data_encoded.drop("Price", axis=1)
y = data_encoded["Price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)




<div class="alert alert-block alert-danger">
<b> Comentário do revisor: </b> <a class="tocSkip"></a>

- precisa descartar features irrelevantes: datecrawled, registratinomonth, datecreated, etc. precisa usar features que realmente influenciam o preço do carro
- algumas features como vehicle type, fuel type,  convém transformar para one hot encoding usando https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
</div>


In [ ]:
%%time
# Regressão Linear
lr_model = LinearRegression()
lr_params, lr_mse = find_best_hyperparameters(
    lr_model, X_train, y_train
)


Calibrando hiperparâmetros para LinearRegression...
Melhores parâmetros: {'fit_intercept': True}
MSE na validação cruzada: 10013524.15
CPU times: user 53.3 s, sys: 21.8 s, total: 1min 15s
Wall time: 40.1 s


In [ ]:
lr_mse, lr_r2 = evaluate_final_model(
    lr_model, lr_params, X_train, y_train, X_test, y_test
)


MSE no conjunto de teste: 10347762.20
R2 no conjunto de teste: 0.50


*Observações: MSE de 10347762.20 e R2 de 0.50 indicam que o modelo é pouco eficiente em predizer os dados de preço com as features disponíveis. Não obstante, o Wall time de 40.1 s s demonstra que a regressão linear possui um treinamento relativamente rápido!*

In [ ]:
%%time
# Floresta Aleatória
rf_model = RandomForestRegressor(random_state=42)

rf_params, rf_mse = find_best_hyperparameters(
    rf_model, X_train, y_train
)



Calibrando hiperparâmetros para RandomForestRegressor...


In [ ]:

rf_mse, rf_r2 = evaluate_final_model(
    rf_model, rf_params, X_train, y_train, X_test, y_test
)


*Observações: MSE de 5645874.13 e R2 de 0.7273 indicam que o modelo é muito mais eficiente em predizer os dados de preço com as features disponíveis, quando comparado com a Regressão Linear. Não obstante, o Wall time de 24min 57s demonstra que a floresta aleatória possui um treinamento muito mais demorado!*

In [ ]:
%%time
X_lgbm = data_encoded.drop("Price", axis=1)
y_lgbm = data_encoded["Price"]
X_lgbm_train, X_lgbm_test, y_lgbm_train, y_lgbm_test = train_test_split(
    X_lgbm, y_lgbm, test_size=0.2, random_state=42
)

# LightGBM Gradient Boosting
lgbm_model = LGBMRegressor(
    random_state=42, 
    verbosity=-1)


lgbm_params, lgbm_mse = find_best_hyperparameters(
    lgbm_model, X_train, y_train
)



In [ ]:
lgbm_mse, lgbm_r2 = evaluate_final_model(
    lgbm_model, lgbm_params, X_train, y_train, X_test, y_test
)


*Observações: MSE de 5956336.26 e R2 de 0.7123 indicam que o o gradient boosting alcança resultados similares aos da floresta aleatória, consumindo uma fração do tempo de treinamento, com Wall time de 45.6s!* 



<div class="alert alert-block alert-danger">
<b> Comentário do revisor: </b> <a class="tocSkip"></a>

- há um erro no treinamento: usar o conjunto de teste para calibrar hiperparâmetro e reportar resultado final
- o conjunto de teste só pode ser usado após calibração de hiperparâmentro. 
- Para fins de calibração, você pode usar treino e validação ou usar gridsearch/randomsearch no treino; em ambos os casos, o teste só é usado após calibração.
- Deixo essa referência para gridsearch: https://www.geeksforgeeks.org/machine-learning/hyperparameter-tuning/ e esta para train-val-test split: https://www.machinelearningmastery.com/training-validation-test-split-and-cross-validation-done-right/
</div>

# Checklist

Digite 'x' para verificar. Em seguida, pressione Shift + Enter.

- [x]  O Jupyter Notebook está aberto
- [ ]  O código está livre de erros
- [ ]  As células com o código foram organizadas em ordem de execução
- [ ]  Os dados foram baixados e preparados
- [ ]  Os modelos foram treinados
- [ ]  A análise de velocidade e qualidade dos modelos foi realizada